# タスク 6: Bedrock モデルと LangGraph の統合

このノートブックでは、アクションの順序を決定し、エージェントが使用できるツールを使用してアクションを実装する計画および実行エージェントの使用方法を学習します。

特定のアプリケーションでは、ユーザーの質問に答えるために、言語モデルやさまざまなユーティリティへの適応可能な呼び出しシーケンスが必要です。Langchain エージェントインターフェイスは柔軟性があり、外部ツールを LLM の推論と統合できます。エージェントは、ユーザーの入力に基づいて使用するツールを選択できます。エージェントは複数のツールを使用でき、1 つのツールの出力を次のツールの入力として利用できます。

## タスク 6.1: 環境の設定

このタスクでは、環境を設定します。

In [ ]:
#create a service client by name using the default session.
import math
import numexpr
import json
import datetime
import sys
import os

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

次に、LangChain の ChatBedrock クラスのインスタンスを作成します。これにより、Amazon Bedrock でホストされている会話型 AI モデルと対話できるようになります。

In [ ]:
#create an instance of the ChatBedrock
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id=model_id , 
    client=bedrock_client)

In [ ]:
#invoke model
chat_model.invoke("AWSとは何ですか？一文で答えてください")

## タスク 6.2: 言語モデル フレームワークにおける推論と行動の相乗効果

このタスクでは、ReAct フレームワークにより、大規模な言語モデルが外部ツールと対話して追加情報を取得し、より正確で事実に基づいた応答を得ることができます。

大規模な言語モデルは、推論の説明とタスク固有の応答の両方を交互に生成できます。

推論の​​説明を生成することで、モデルはアクションプランを推論、監視、修正し、予期しないシナリオを処理することさえできます。アクションステップでは、モデルが知識ベースや環境などの外部ソースと対話して情報を取得できます。

In [ ]:
from langchain_core.tools import tool

次のセルでは、LangChain フレームワーク内のツールとして機能し、クエリで指定された製品の価格を前のタスクで作成された `sales.csv` ファイルから取得する関数 `get_product_price` を定義します。これは、LangChain フレームワークで動作するようにツールを設計する方法を示す簡単な実装です。

In [ ]:
@tool
def get_product_price(query:str):
    "Useful when you need to lookup product price"
    import csv
    prices = {}
    try:
        file=open('sales.csv', 'r')
    except Exception as e:
        return ("価格を調べることができません " + query)
    reader = csv.DictReader(file)
    for row in reader:
        prices[row['product_id']] = row['price']
    file.close()
    qstr=query.split("\n")[0].strip()
    try:
            return ("製品: "+qstr+" 価格: "+prices.get(qstr)+"\n")
    except:
            return ("製品: "+qstr+" の情報はありません"+"\n")

次のセルでは、LangChain フレームワーク内のツールとして機能する関数 `calculator` を定義します。このツールにより、言語モデルは Python の numexpr ライブラリを使用して指定された式を評価することで数学計算を実行できます。このツールは、式が無効なケースを処理するように設計されています。その場合、ツールはモデルに計算へのアプローチを再考するように要求します。

In [ ]:
@tool
def calculator(expression: str) -> str:
    """このツールを使用すると、1 行の数式を含む数学の問題を解くことができます。
       言葉ではなく、数式表記を使用してください。
       例:
        "5*4" for "5 times 4"
        "5/4" for "5 divided by 4"
    """
    try:
        return str(
            numexpr.evaluate(
            expression.strip(),
            global_dict={},  
            local_dict={} # add math constants, if needed
            )
        )
    except Exception as e:
        return "この計算に対するアプローチを再考してください"

In [ ]:
tools = [get_product_price, calculator]

次のセルでは、ヘルパー関数を実行してトレース出力をファイルに出力します。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
def output_trace(element:str, trace, node=True):
    global trace_handle
    if trace_enabled:
        print(datetime.datetime.now(),file=trace_handle)
        print(("Node: " if node else "Edge: ")+ element, file=trace_handle)
        if element == "ask_model_to_reason (entry)":
            for single_trace in trace:
                print(single_trace, file=trace_handle)
        else:
            print(trace, file=trace_handle)
        print('----', file=trace_handle)
        
def consolidate_tool_messages(message):
    tool_messages=[]
    for msg in message:
        if isinstance(msg, ToolMessage):
            tool_messages.append(msg)
    return tool_messages

## タスク 6.3: エージェントグラフの構築

このタスクでは、外部ツールと対話できる会話型 AI システムのエージェントグラフを作成します。エージェントグラフは、会話のフローとツールとの対話を定義するステートマシンです。

次のセルでは、入力に基づいて状態を更新する関連関数を持つノードを定義します。エッジを使用してノードを接続します。グラフは 1 つのノードから次のノードに遷移します。条件付きエッジを組み込んで、特定の条件に基づいてグラフを異なるノードにルーティングします。最後に、エージェントグラフをコンパイルして実行の準備をし、定義どおりに遷移と状態の更新を処理します。

In [ ]:
from typing import Literal
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# ToolNode is a prebuilt component that runs the tool and appends the tool result to the messages 
tool_node = ToolNode(tools)

# let the model know the tools it can access
model_with_tools = chat_model.bind_tools(tools)
    
# The following function acts as the conditional edge in the graph.
# The next node could be the tools node or the end of the chain.
def next_step(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        output_trace("next_step: Proceed to tools",last_message, node=False)
        return "tools"
    output_trace("next_step: Proceed to end",last_message, node=False)
    return "__end__"

#.The following node function invokes the model that has information about the available tools
def ask_model_to_reason(state: MessagesState):
    messages = state["messages"]
    output_trace("ask_model_to_reason (entry)", consolidate_tool_messages(messages))
    try:
        response = model_with_tools.invoke(messages)
    except Exception as e:
        output_trace("ask_model_to_reason", messages)
        output_trace("ask_model_to_reason", "Exception: "+str(e))
        return {"messages": [messages.append("Unable to invoke the model")]}
    output_trace("ask_model_to_reason (exit)", response)
    return {"messages": [response]}


agent_graph = StateGraph(MessagesState)

# Describe the nodes. 
# The first argument is the unique node name, and the second argument is the 
# function or object that will be called when the node is reached
agent_graph.add_node("agent", ask_model_to_reason)
agent_graph.add_node("tools", tool_node)

# Connect the entry node to the agent for the graph to start running
agent_graph.add_edge("__start__", "agent")

# Once the graph transitions to the tools node, the graph will transition to the agent node
agent_graph.add_edge("tools", "agent")

# The transition out of the agent node is conditional. 
# If the output from ask_model_to_reason function included a call to the tools, call the tool; 
# otherwise end the chain 
agent_graph.add_conditional_edges(
    "agent",
    next_step,
)

# Compile the graph definition so that it can run

react_agent = agent_graph.compile()

次に、コンパイルされたグラフを視覚化します。点線で示されているように、エージェントノードからの遷移が条件付きであることを確認します。

In [ ]:
from IPython.display import Image, display

try:
    display(Image(react_agent.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

次のセルでは、グラフ出力を印刷するためのヘルパー関数を実行します。

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

次に、前のノートブックで作成した sales.csv ファイルから、エージェントに製品の価格について尋ねたい質問を 1 つ以上追加します。

In [ ]:
#list of questions
questions=[]
questions.append("P002 を 3 台、P003 を 5 台購入するといくらかかりますか?")
#questions.append("200 ドルで P010 を何台購入できますか?")
#questions.append("200ドルでP003を3台購入できますか？購入できない場合、3台購入するのにあといくらかかりますか?")
#questions.append("価格が 8% 上昇しました。価格が上昇する前、140 ドルで P003 を何個購入できましたか? 価格が上昇した後は、何個購入できますか? 端数購入はできません。")

推論に含まれる手順を理解するには、トレースを有効にします。ただし、上記のリストの **1 つの質問以外をすべてコメントアウト**して、トレース出力を管理しやすい状態にしておきます。または、トレースを無効にして、すべての質問を実行することもできます。

In [ ]:
trace_enabled=True

if trace_enabled:
    file_name="trace_"+str(datetime.datetime.now())+".txt"
    trace_handle=open(file_name, 'w')

次のセルでは、上記のリストにある質問でエージェントを呼び出します。

In [ ]:
system_message="次の質問にできる限り答えてください。答えをでっち上げないでください。段階的に考えてください。中級レベルの数学計算を自分で行わないでください。数学計算には提供されている計算ツールを使用してください。"

for q in questions:
    inputs = {"messages": [("system",system_message), ("user", q)]}
    config={"recursion_limit": 15}
    print_stream(react_agent.stream(inputs, config, stream_mode="values"))
    print("\n"+"================================ Answer complete ================================="+"\n")

if trace_enabled:
    trace_handle.close()